1. Align with accuracy instead of epoch
- One epoch that closest to 80% accuracy on PHO
2. Plot individual “network” difference beta over grid
- Taraban : y~lm(freq x cons)
- IMG-HS04 : y~lm(fxcximg)
- Nonword Glushko overall: just acc
3. Big stat model on the entire grid
- Y ~ batch_size  or epsilon check same dimensions or not… 
- y ~ lm/lmer(batch_size  or epsilon * stimprop)  | testset x
4. Also summarize DoL within the same grid [raw, same epoch at 1]
- P: intact, OP, OSP
- S: intact, OS, OPS


# Get merged data

In [ ]:
import meta
import os
import pandas as pd
import numpy as np
import altair as alt
from itertools import chain
from tqdm import tqdm

In [ ]:
class Batch:
    """
    
    
    """

    def __init__(self, batch_name: str, tf_root: str = None):
        self.batch_name = batch_name
        self.json = os.path.join("models", batch_name, "batch_config.json")
        self.tf_root = tf_root if tf_root else "./"
        self.cfg_df = self.parse_batch_config()
        self.code_names = self.cfg_df.code_name.unique().tolist()
        self.df = self.parse_df(['train_r100_triangle.csv'])

    def mount_testset(self, csv: list):
        self.df = self.parse_df(csv)

    def subset_df(self, code_name:str=None, epoch:int=None, output_name:str=None, timetick:list=None, cond:list=None):
        """Return a subset of the dataframe."""
        df = self.df
        df = df.loc[df.code_name == code_name] if code_name is not None else df
        df = df.loc[df.epoch == epoch] if epoch is not None else df 
        df = df.loc[df.output_name == output_name] if output_name is not None else df
        df = df.loc[df.timetick.isin(timetick)] if timetick is not None else df
        df = df.loc[df.cond.isin(cond)] if cond is not None else df
        return df

    def subset_by_epoch_dict(self, sel_epoch:dict):
        """Return a subset of the dataframe using a epoch dictionary.
        args:
            sel_epoch: dictionary of epochs to select with k=code_name, v=epoch
        """
        dfs = [self.subset_df(code_name=k, epoch=v) for k, v in sel_epoch.items()]
        return self.concat_dfs(dfs)

    def parse_batch_config(self):
        df = meta.batch_json_to_df(self.json, tf_root=self.tf_root)
        assert (
            self.batch_name == "task_effect"
        )  # Just in case I forgot to change below line in other batches
        df["train_task"] = [
            "OP",
            "OS",
            "Triangle",
        ] * 12  # Caution: this is a hack to get around list type config, only works for this batch
        return df[["code_name", "batch_size", "learning_rate", "train_task"]]

    def parse_df(self, csv: list) -> pd.DataFrame:
        files = chain.from_iterable([self.get_eval_file_names(x) for x in csv])
        df = self.merge_from_file_names(files)
        return df.merge(self.cfg_df, on="code_name", how="left")


    def get_eval_file_names(self, csv_name: str) -> list:
        """Return a list of dataframes from a list of csvs."""
        return [
            os.path.join(
                self.tf_root, "models", self.batch_name, code_name, "eval", csv_name
            )
            for code_name in self.code_names
        ]

    def find_code_name(self, criteria: dict) -> str:
        """Return a code_name from a dictionary of criteria."""
        mask = None
        for k, v in criteria.items():
            hit = (self.cfg_df[k].isin(v)).to_list()
            mask = hit if mask is None else (a & b for a, b in zip(mask, hit))

        return self.cfg_df.code_name.loc[mask].tolist()

    def find_epoch_by_acc(self, code_name: str, acc: float) -> int:
        """Return an epoch number from an accuracy."""
        df = self.df.loc[self.df.code_name == code_name]
        df = df.loc[df.output_name == 'pho'] # PHO as criteria
        df = df.loc[df.timetick.isin(range(8, 13))] # Selecting 8-12 ticks
        df = df.groupby('epoch').mean().reset_index() # Group by epoch
        idx = self.find_nearest(df.acc, acc) # Find nearest accuracy
        return df.iloc[idx,].epoch # Return epoch
        
    @staticmethod
    def merge_from_file_names(filenames: list) -> list:
        """Merge a list of dataframes into one."""
        dfs = [pd.read_csv(f) for f in filenames]
        return Batch.concat_dfs(dfs)

    @staticmethod
    def concat_dfs(dfs: list) -> pd.DataFrame:
        """Return a dataframe from a list of dataframes."""
        return pd.concat(dfs, ignore_index=True).reset_index(drop=True)

    @staticmethod
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx

    @staticmethod
    def get_acc_based_df(self, acc: float) -> pd.DataFrame:
        """Return a dataframe of accuracy for a code_name."""

        df = self.df.loc[self.df.code_name == code_name].copy()
        # Subset to nearest accuracy epoch
        sel_epoch = self.find_epoch_by_acc(code_name, 0.8)
        df = df.loc[df.epoch == sel_epoch]
        return df


    


b = Batch("task_effect")


# Find epoch that are closest to 80% accuracy in each network

- Define by train_r100 testset
- at 8-12 ticks

In [ ]:
b.mount_testset(['taraban_triangle.csv'])

In [ ]:
sel_epoch = {x:b.find_epoch_by_acc(x, 0.8) for x in tqdm(b.code_names)}
print(sel_epoch)

In [ ]:
df = b.subset_by_epoch_dict(sel_epoch)

In [ ]:
sel_conds = [
    "High-frequency exception",
    "Regular control for High-frequency exception",
    "Low-frequency exception",
    "Regular control for Low-frequency exception",
    ]

df = df.loc[df.cond.isin(sel_conds)] # Select conditions
df = df.loc[df.output_name == 'pho'] # P output
df = df.loc[df.timetick.isin(range(8, 13))] # timetick 8-12
df = df.loc[df.train_task == 'Triangle'] 

In [ ]:
df.to_csv('issues/0_batchsize_lr/taraban80.csv')

In [ ]:
df = df.groupby(['batch_size', 'learning_rate', 'code_name', 'epoch', 'cond']).mean().reset_index()

# Try inferential statistics

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats.mstats import zscore

In [ ]:

m = smf.glm(formula='zscore(acc) ~ zscore(learning_rate) + zscore(batch_size) + C(reg)  + C(freq)', data=mdf).fit()
print(m.summary())

In [ ]:
m.params

In [ ]:
df.columns

In [ ]:
alt.Chart(df).mark_text().encode(
    x='learning_rate:O',
    y='batch_size:O',
    text='mean(acc):Q'
    ).properties(width=500, height=500)

# Taraban dataset

In [ ]:
b.mount_testset(['taraban_triangle.csv'])
df = b.subset_by_epoch_dict(sel_epoch)

In [ ]:
sel_conds = ["High-frequency exception",
            "Regular control for High-frequency exception",
            "Low-frequency exception",
            "Regular control for Low-frequency exception",
            ]

df = df.loc[df.cond.isin(sel_conds)] # Select conditions
df = df.loc[df.output_name == 'pho'] # P output
df = df.loc[df.timetick.isin(range(8, 13))] # timetick 8-12
df = df.loc[df.train_task == 'Triangle'] 
    
df["freq"] = df.cond.apply(
    lambda x: "High"
    if x
    in ("High-frequency exception", "Regular control for High-frequency exception")
    else "Low"
)
df["reg"] = df.cond.apply(
    lambda x: "Regular" if x.startswith("Regular") else "Exception"
)


df = df[['batch_size', 'learning_rate', 'code_name', 'epoch', 'timetick', 'freq', 'reg', 'word', 'acc', 'sse']]
df.to_csv(os.path.join('issues', '0_batchsize_lr', 'taraban80.csv'))
mdf = df.groupby(['batch_size', 'learning_rate', 'freq', 'reg']).mean().reset_index()

In [ ]:
mdf = df.groupby(['batch_size', 'learning_rate', 'code_name', 'freq', 'reg']).mean().reset_index()

def plot_taraban(df, metric: str = 'acc'):
    metric_specific_scale = alt.Scale(domain=(0, 1)) if metric == "acc" else alt.Scale()
    return alt.Chart(df).mark_line().encode(
            x=alt.X("freq:N", scale=alt.Scale(reverse=True)),
            y=alt.Y(f"mean({metric}):Q", scale=metric_specific_scale),
            row="batch_size:O",
            column="learning_rate:O",
            color="reg:N",
        ).properties(width=150, height=150)


plot_taraban(mdf, 'acc').save('taraban_acc.html')

In [ ]:
df.to_csv('taraban_at_80.csv')

### Select PHO accuracy at last time tick

In [ ]:
import altair as alt

heat = alt.Chart(acc_df).mark_rect().encode(
    x='learning_rate:O',
    y=alt.Y('batch_size:O'),
    color=alt.Color('acc:Q', scale=alt.Scale(domain=[0, 1])),
)

heat.mark_text().encode(
    text=alt.Text('acc:Q', format='.2f')
).properties(width=300, height=300).configure_text(fontSize=20)


### Selected epoch

In [ ]:
heat.mark_text().encode(
    text=alt.Text('epoch:Q', format='.0f')
).properties(width=300, height=300).configure_text(fontSize=20)